# RNN

#### Types of RNN Architectures

Vanilla RNN: Basic RNN but struggles with long sequences (vanishing gradient problem).

LSTM (Long Short-Term Memory): Handles long-term dependencies effectively.

GRU (Gated Recurrent Unit): Similar to LSTM but computationally faster

#### Key Concepts in RNNs

Hidden State: Stores information about previous inputs.

Vanishing Gradient Problem: Gradients shrink during backpropagation, leading to poor learning in long sequences.

LSTM/GRU: Introduce gates to selectively remember and forget information.

Embedding Layer: Converts words into dense vectors to capture meaning.

Dropout: Prevents overfitting by randomly dropping connections.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import IMDB
from torchtext.data import Field, BucketIterator
import spacy


## Step 2: Preprocess Dataset

In [ ]:
TEXT = Field(tokenize='spacy', tokenizer_language='en_core_web_sm', lower=True)
LABEL = Field(sequential=False, use_vocab=False)

train_data, test_data = IMDB.splits(TEXT, LABEL)

# Build vocabulary with embeddings
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.100d")


## Step 3: Create DataLoader

In [ ]:
train_loader, test_loader = BucketIterator.splits(
    (train_data, test_data), batch_size=64, sort_within_batch=True
)


## Step 4: Define RNN Model

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, dropout=0.5)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, cell) = self.rnn(embedded)
        return self.fc(hidden[-1])


## Step 5: Train the Model

In [ ]:
model = RNN(len(TEXT.vocab), 100, 256, 1)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

num_epochs = 5
for epoch in range(num_epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label.float())
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


## Evaluate the Model

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        outputs = model(batch.text).squeeze(1)
        predicted = torch.round(torch.sigmoid(outputs))
        correct += (predicted == batch.label).sum().item()
        total += batch.label.size(0)

print(f"Accuracy: {100 * correct / total:.2f}%")
